# TP n°7 : Plus longue sous-séquence commune

Soit $l=[l_0,\dots,l_{n-1}]$ une liste. On appelle sous-séquence de la liste $l$ toute liste obtenue à partir de $l$ en supprimant éventuellement certains éléments et en laissant les autres dans l'ordre. Autrement dit, une sous-séquence de $l$ est une liste de la forme $[l_{\varphi(0)};\dots;l_{\varphi(k-1)}]$ avec $0 \leqslant \varphi(0)<\dots<\varphi(k-1) \leqslant n-1$.
#### Question 1

Écrire une fonction `est_sous_sequence` qui prend en argument deux listes $l_1$ et $l_2$ et qui teste si $l_1$ est une sous-séquence de $l_2$.

In [1]:
let rec est_sous_sequence l1 l2 =
  match l1, l2 with
  |[], _ -> true
  | _, [] -> false
  | t1::q1, t2::q2 ->
     if t1 = t2
     then est_sous_sequence q1 q2
     else est_sous_sequence l1 q2            
;;

val est_sous_sequence : 'a list -> 'a list -> bool = <fun>


#### Question 2
Soit $l$ une liste formée de $n$ éléments distincts. Combien existe-il de sous-séquences de $l$ ?


Il y en a $2^n$.


#### Question 3
Écrire une fonction récursive `sous_sequences`  prenant en argument une liste et renvoyant la liste de ses sous-séquences.

In [2]:
let rec sous_sequences lst =
  (* ajoute_ou_pas renvoie la liste de listes obtenue en ajoutant ou non x  à chaque liste de la liste de listes ll ; 
  la longueur de la liste est donc doublée *)
  let rec ajoute_ou_pas x ll =
    match ll with
    | [] -> []
    | lt::llq -> lt::(x::lt)::(ajoute_ou_pas x llq)
  in
  match lst with
  | [] -> [[]]
  | t::q ->
     let ss_sq_q = sous_sequences q in
     ajoute_ou_pas t ss_sq_q
;;

let rec sous_sequences lst =
  match lst with
  | [] -> [[]]
  | t::q ->
     let ss_sq_q = sous_sequences q in
     ss_sq_q@(List.map (fun l -> t::l) ss_sq_q)
;;

val sous_sequences : 'a list -> 'a list list = <fun>


val sous_sequences : 'a list -> 'a list list = <fun>


Soient $x=[x_0,\dots,x_{n-1}]$  et $y=[y_0,\dots,y_{m-1}]$ deux listes formées d'éléments du même type. On cherche à déterminer la longueur maximale d'une sous-séquence commune à $x$ et $y$ et à trouver une sous-séquence commune de cette longueur. On parle alors de plus longue sous-séquence commune (attention, il n'y a pas forcément unicité : ne pas parler de *la* plus longue sous-séquence commune).

Une première façon de résoudre le problème est de chercher toutes les sous-séquences communes afin de trouver l'une des plus longues.
#### Question 4
En utilisant les questions précédentes, écrire une fonction `plssc1` qui renvoie une plus longue sous-séquence commune de deux listes.

In [3]:
let plssc1 l1 l2 =
  let ssl1 = sous_sequences l1 in
  (* aux renvoie la plus longue sous-séquence commune entre plssc de longueur lgr et les sous-séquences de reste1 *)
  let rec aux reste1 lgr plssc =
    match reste1 with
    | [] -> plssc
    | t::q ->
       (* Est-ce que t est une ss de l2 ?*)
       let lgrt = List.length t in
       if est_sous_sequence t ssl2 && lgr < lgrt
       then aux q lgrt t
       else aux q lgr plssc
  in
  aux ssl1 0 []
;;

error: compile_error

Cet algorithme est en temps exponentiel donc plutôt mauvais. On va essayer de concevoir un algorithme plus rapide.

On note $L(i,j)$ la longueur d'une plus longue sous-séquence commune de $[x_0,\dots,x_{i-1}]$ et $[y_0,\dots,y_{j-1}]$.

#### Question 5
Montrer les relations suivantes :

- $\forall i \in  ⟦0,n⟧, L(i,0)=0$ et $\forall j \in ⟦0,m⟧, L(0,j)=0$
- Si $i>0$ et $j>0$, alors 
$\displaystyle L(i,j) = \left\lbrace \begin{array}{ll} 1+L(i-1,j-1) & \text{ si } x_{i-1}=y_{j-1} \\
	\max \left( L(i-1,j) , L(i,j-1) \right) & \text{ sinon} \end{array}\right.$

#### Question 6
En déduire une fonction récursive `lplssc1` qui calcule la longueur d'une plus longue sous-séquence commune de deux listes. <br/>
	*On pourra remarquer que la longueur d'une plus longue sous-séquence commune de deux listes est égale à la longueur d'une plus longue sous-séquence commune des miroirs des deux listes.*

In [4]:
let rec lplssc1 l1 l2 =
  match l1, l2 with
  | [], _ -> 0
  | _, [] -> 0
  | t1::q1, t2::q2 ->
     if t1 = t2
     then 1 + lplssc1 q1 q2
     else max (lplssc1 l1 q2) (lplssc1 q1 l2)
;;

val lplssc1 : 'a list -> 'a list -> int = <fun>


On pourra montrer que le nombre d'appels récursifs dans le pire des cas effectués par cette fonction peut être minoré par une exponentielle en $\min (n,m)$.
#### Question 7
Lorsque $n=m=3$, combien de fois calcule-t-on $L(1,0)$ dans le pire des cas ?


Dans le pire des cas, $L(3,3)$ nécessite :
- le calcul de $L(2, 3)$, qui nécessite
    - le calcul de $L(1,2)$, qui nécessite un appel à $L(0,1)$ et un appel à $L(1,1)$ qui lui-même appelle $L(0,1)$ et $L(1,0) $;
    - le calcul de $L(2,2)$ qui nécessite un appel à $L(1,2)$, pour lequel on calcule une fois $L(1,0)$, et un appel à $L(2,1)$ qui lui-même appelle $L(1,1)$ (pour lequel on calcule une fois $L(1,0)$) et un appel à $L(2,0)$.
- le calcul de $L(3,2)$ qui nécessite :
    - celui de $L(2,2)$, pour lequel on calcule deux fois $L(1,0)$
    - celui de $L(3,1)$, qui appelle $L(2,1)$ (pour lequel on calcule une fois $L(1,0)$) et $L(3,0)$.
    
Finalement, dans le pire des cas, $L(1,0)$ est calculé six fois lorsqu'on calcule $L(3,3)$.


De nombreuses valeurs sont calculées plusieurs fois. Pour éviter cela, on conçoit un algorithme de programmation dynamique pour le calcul des $L(i,j)$, en stockant leurs valeurs dans une matrice. 

Dans cette partie, on pourra convertir les listes en tableaux à l'aide de la fonction `Array.of_list`

#### Question 8
Indiquer dans quel ordre remplir la matrice. Une fois cette matrice calculée, comment déterminer la longueur d'une plus longue sous-séquence commune ?


On peut remplir la matrice par $i$ croissant puis $j$ croissant ou inversement.
La longueur d'une plus longue sous-séquence commune est alors égale au coefficient $(n,m)$.


#### Question 9
Écrire une fonction `lplssc2` qui calcule la longueur d'une plus longue sous-séquence commune de deux listes en implémentant cette méthode. Quelle est sa complexité ?

*Pour créer la matrice, on utilisera la fonction `Array.make_matrix`*

In [5]:
let lplssc2 l1 l2 =
  let t1 = Array.of_list l1 in
  let t2 = Array.of_list l2 in
  let len1 = Array.length t1 in
  let len2 = Array.length t2 in  
  let mat_L = Array.make_matrix (len1 + 1) (len2 + 1) 0 in
  for i = 1 to len1 do
    for j = 1 to len2 do
      if t1.(i-1) = t2.(j-1)
      then
        mat_L.(i).(j) <- 1 + mat_L.(i-1).(j-1)
      else
        mat_L.(i).(j) <- max mat_L.(i-1).(j) mat_L.(i).(j-1)
    done
  done;
  mat_L.(len1).(len2)
;;

val lplssc2 : 'a list -> 'a list -> int = <fun>


#### Question 10
Comment retrouver une plus longue sous-séquence commune à partir de la matrice des $L(i,j)$ ? 

#### Question 11
Écrire une fonction `plssc2` qui calcule une plus longue sous-séquence commune.

In [6]:
let plssc2 l1 l2 =
  let t1 = Array.of_list l1 in
  let t2 = Array.of_list l2 in
  let len1 = Array.length t1 in
  let len2 = Array.length t2 in  
  let mat_L = Array.make_matrix
        (len1 + 1)
        (len2 + 1)
        0 in
  for i = 1 to len1 do
    for j = 1 to len2 do
      if t1.(i-1) = t2.(j-1)
      then
    mat_L.(i).(j) <- 1 + mat_L.(i-1).(j-1)
      else
    mat_L.(i).(j) <- max mat_L.(i-1).(j) mat_L.(i).(j-1)           
    done
  done;
  let rec reconstruit i j fin =
    match mat_L.(i).(j) with
    | 0 -> fin
    | _ ->
       if t1.(i-1) = t2.(j-1)
       then reconstruit (i-1) (j-1) (t1.(i-1)::fin)
       else
     if mat_L.(i).(j) =  mat_L.(i-1).(j)
     then reconstruit (i-1) j fin
     else reconstruit i (j-1) fin
  in
  reconstruit len1 len2 []         
;;

val plssc2 : 'a list -> 'a list -> 'a list = <fun>


In [7]:
plssc2 [1;2;3;42;4;5;6] [2;4;42;6;8;5]

- : int list = [2; 42; 5]


#### Question 12
L'adapter pour qu'elle donne toutes les plus longue sous-séquence commune.

In [8]:
let toutes_les_plssc l1 l2 =
  let t1 = Array.of_list l1 in
  let t2 = Array.of_list l2 in
  let len1 = Array.length t1 in
  let len2 = Array.length t2 in  
  let mat_L = Array.make_matrix
        (len1 + 1)
        (len2 + 1)
        0 in
  for i = 1 to len1 do
    for j = 1 to len2 do
      if t1.(i-1) = t2.(j-1)
      then
    mat_L.(i).(j) <- 1 + mat_L.(i-1).(j-1)
      else
    mat_L.(i).(j) <- max mat_L.(i-1).(j) mat_L.(i).(j-1)           
    done
  done;
  (* La fonction suivante prend en argument :
      - les entiers i et j correspondant à la position courante dans la matrice
      - le suffixe des plssc en cours de construction
      - une référence sur une liste qui contient les plssc déjà trouvées
      - un booléen indiquant s'il est autorisé de se déplacer à gauche : 
        pour éviter les doublons, si on se déplace vers le haut, on ne se déplacera 
        pas à gauche tant qu'un élément de la plssc n'aura pas été trouvé
    Cette fonction agit par effet de bord, en ajoutant des éléments à la référence liste
  *)
  let rec reconstruit i j fin liste gauche =
    match mat_L.(i).(j) with
    | 0 -> liste := fin::!liste
    | _ ->
       if t1.(i-1) = t2.(j-1)
       then reconstruit (i-1) (j-1) (t1.(i-1)::fin) liste true;
       if mat_L.(i).(j) =  mat_L.(i-1).(j)
       then reconstruit (i-1) j fin liste false;
       if gauche && mat_L.(i).(j) =  mat_L.(i).(j-1)
       then reconstruit i (j-1) fin liste true
  in
  let liste_ssc = ref [] in
  reconstruit len1 len2 [] liste_ssc true;
  !liste_ssc
;;

val toutes_les_plssc : 'a list -> 'a list -> 'a list list = <fun>


In [9]:
toutes_les_plssc [1;2;3;42;4;5;6] [2;4;42;6;8;5]

- : int list list = [[2; 4; 6]; [2; 42; 6]; [2; 4; 5]; [2; 42; 5]]
